# ProbTTR Bayes classifier

In [1]:
from probttrtypes import Type,VarType,PRound,MeetType
from utils import show

## Random variables in TTR

In ... Staffan Larsson has introduced a formulation of random variables as a TTR type.  In probabilistic `pyttr` we  create such a type as an instance of the class `VarType` (variable type).  We create a variable type by `V = VarType(types)` where `types` is a list of types, known as the value types.  The idea is that an object will be of type `V` with either probability 1 or 0.  For any witness for `V`, we will obtain a distribution of probabilities over the types in `types`, that is, the sum of the probabilities that the witness is of type `T` for all the types in `types` will be 1.  Below we define an abstract example.

In [2]:
A1 = Type()
A2 = Type()
A3 = Type()
A1.judge('a',.3)
A2.judge('a',.3)
A3.judge('a',.4)
V1 = VarType([A1,A2,A3])
print(show(V1.query('a')))

1.0


In order to query the probability that an object is of type `T` with respect to a variable type `V`, we use the  method `query_v` as in `query_v('a',T)`.

In [3]:
for T in [A1,A2,A3]:
    print(show(V1.query_v('a',T)))

0.3
0.3
0.4


In the example above the result returned by `V1.query_v('a',T)` for any of the types on which `V1` is defined is the same as the result returned by `T.query('a')` because we declared the probabilities to form a distribution.  In the theoretical presentation of random variables in TTR it is required that the probabilities computed independently of the variable type are the same as the probabilities computed with respect to the variable type.  However, in `pyttr` we allow the variable to distribute arbitrary probability mass so that a distribution is formed.  If `V` is a variable type based on the types `[T1,...,Tn]`, then `V.query_v(a,Ti)` returns

>$\displaystyle{\frac{\texttt{Ti.query(a)}}{\Sigma_{T\in\texttt{[T1,...,Tn]}}\texttt{T.query(a)}}}$ 

This guarantees that 

>$\displaystyle{\{\texttt{Ti.query}\_\texttt{v(a)}\mid \texttt{Ti}\in\texttt{T1,...,Tn}\}}$

is a probability distribution adding to 1.

The effect of this intuitively is to change the probability of something being a witness type  depending on what other types you are currently comparing it with.  This might be one way of dealing with examples like the difference in redness of faces, wine, coats, apples etc.

In the example below we consider an example of an apple which as an object may be considered to be greenish yellow.  However, in a classification of apples as being either green or red it would be considered to be definitely green.

In [4]:
Green = Type()
Yellow = Type()
Red = Type()
AppleVar = VarType([Green,Red])
Green.judge('a1',.6)
Yellow.judge('a1',.4)
Red.judge('a1',0)
print(show(AppleVar.query_v('a1',Green)))
print(show(AppleVar.query_v('a1',Red)))

1.0
0.0


An object is not a witness for a variable type if it has zero probability of being a witness for all the value types, that is, the types, `T1,...,Tn`, on which the variable type is based.

In [5]:
A1.judge('b',0)
A2.judge('b',0)
A3.judge('b',0)
print(show(V1.query('b')))

0.0


An object will be a witness of the variable type if it has any non-zero probability of being a witness for any of the value types.  (Here we use `PRound`, an adaptation to probability intervals of Python's in-built function `round`, to correct an error which occurs because we are using floating point numbers.)

In [6]:
A1.judge('c',.1)
print(show(PRound(V1.query('c'))))

1.0


This has the consequence that the value types are subtypes of the variable type.

In [7]:
A1.subtype_of(V1)

True

If we query an object for which we have no judgements for any of the value types, "Don't know" ([0,1] or <=1) is returned.

In [8]:
print(show(V1.query('d')))

<=1.0


The value types within a variable type preclude each other, that is, it is not possible for something to be a witness (with probability 1) for more than one of the types.  For example, the probability that something will be of the meet type $\texttt{A1}\wedge\texttt{A2}$ for our example `V1` will be 0.

In [9]:
show(V1.query_v('a',MeetType(A1,A2)))

'0.0'

In the current implementation this preclusion need not hold externally to the variable.  Thus if we query `'a'` with respect to `MeetType(A1,A2)` in the example above we get a different result.

In [10]:
show(MeetType(A1,A2).query('a'))

'0.09'

## Non-specific and conditional probabilities

Variable types can also be queried for non-specific and conditional probabilities as usual.

In [11]:
show(PRound(V1.query_nonspec()))

'1.0'

In [12]:
show(V1.query('d',[('d',A2)])) # Should be 1

'1.0'

In [13]:
show(V1.query_v('a',A1,[('a',A2)])) # Should be 0

'0.1764705882352941'

In [14]:
show(V1.query_v('a',A1,[('a',A1)])) # Should be 1

'0.588235294117647'

In [15]:
not True and False

False